In [1]:
!sudo mount -o discard,defaults /dev/sdb /mnt/disks/mount_point

In [2]:
!sudo chmod a+w /mnt/disks/mount_point/

In [6]:
!pip3 install albumentations

    100% |████████████████████████████████| 92kB 2.9MB/s ta 0:00:011
    100% |████████████████████████████████| 634kB 2.1MB/s eta 0:00:01
    100% |████████████████████████████████| 2.1MB 657kB/s eta 0:00:01
  Running setup.py bdist_wheel for albumentations ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/ec/67/12/fe5e79caf7aa6bbdca1506fc66bfd61fc4eae5abb5397b9c08
  Running setup.py bdist_wheel for imgaug ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug


In [7]:
!pip install efficientnet-pytorch

  Stored in directory: /home/jupyter/.cache/pip/wheels/27/56/13/5bdaa98ca8bd7d5da65cc741987dd14391b87fa1a09081d17a
Successfully built efficientnet-pytorch


In [3]:
import os
import sys

sys.path.append('/home/jupyter/.local/lib/python3.5/site-packages')

In [4]:
from __future__ import print_function, with_statement, division
import copy
import os
import torch
from tqdm.autonotebook import tqdm
from torch.optim.lr_scheduler import _LRScheduler
import matplotlib.pyplot as plt


class LRFinder(object):
    """Learning rate range test.
    The learning rate range test increases the learning rate in a pre-training run
    between two boundaries in a linear or exponential manner. It provides valuable
    information on how well the network can be trained over a range of learning rates
    and what is the optimal learning rate.
    Arguments:
        model (torch.nn.Module): wrapped model.
        optimizer (torch.optim.Optimizer): wrapped optimizer where the defined learning
            is assumed to be the lower boundary of the range test.
        criterion (torch.nn.Module): wrapped loss function.
        device (str or torch.device, optional): a string ("cpu" or "cuda") with an
            optional ordinal for the device type (e.g. "cuda:X", where is the ordinal).
            Alternatively, can be an object representing the device on which the
            computation will take place. Default: None, uses the same device as `model`.
        memory_cache (boolean): if this flag is set to True, `state_dict` of model and
            optimizer will be cached in memory. Otherwise, they will be saved to files
            under the `cache_dir`.
        cache_dir (string): path for storing temporary files. If no path is specified,
            system-wide temporary directory is used.
            Notice that this parameter will be ignored if `memory_cache` is True.
    Example:
        >>> lr_finder = LRFinder(net, optimizer, criterion, device="cuda")
        >>> lr_finder.range_test(dataloader, end_lr=100, num_iter=100)
    Cyclical Learning Rates for Training Neural Networks: https://arxiv.org/abs/1506.01186
    fastai/lr_find: https://github.com/fastai/fastai
    """

    def __init__(self, model, optimizer, criterion, device=None, memory_cache=True, cache_dir=None):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.history = {"lr": [], "loss": []}
        self.best_loss = None
        self.memory_cache = memory_cache
        self.cache_dir = cache_dir

        # Save the original state of the model and optimizer so they can be restored if
        # needed
        self.model_device = next(self.model.parameters()).device
        self.state_cacher = StateCacher(memory_cache, cache_dir=cache_dir)
        self.state_cacher.store('model', self.model.state_dict())
        self.state_cacher.store('optimizer', self.optimizer.state_dict())

        # If device is None, use the same as the model
        if device:
            self.device = device
        else:
            self.device = self.model_device

    def reset(self):
        """Restores the model and optimizer to their initial states."""
        self.model.load_state_dict(self.state_cacher.retrieve('model'))
        self.optimizer.load_state_dict(self.state_cacher.retrieve('optimizer'))
        self.model.to(self.model_device)

    def range_test(
        self,
        train_loader,
        val_loader=None,
        end_lr=10,
        num_iter=100,
        step_mode="exp",
        smooth_f=0.05,
        diverge_th=5,
    ):
        """Performs the learning rate range test.
        Arguments:
            train_loader (torch.utils.data.DataLoader): the training set data laoder.
            val_loader (torch.utils.data.DataLoader, optional): if `None` the range test
                will only use the training loss. When given a data loader, the model is
                evaluated after each iteration on that dataset and the evaluation loss
                is used. Note that in this mode the test takes significantly longer but
                generally produces more precise results. Default: None.
            end_lr (float, optional): the maximum learning rate to test. Default: 10.
            num_iter (int, optional): the number of iterations over which the test
                occurs. Default: 100.
            step_mode (str, optional): one of the available learning rate policies,
                linear or exponential ("linear", "exp"). Default: "exp".
            smooth_f (float, optional): the loss smoothing factor within the [0, 1[
                interval. Disabled if set to 0, otherwise the loss is smoothed using
                exponential smoothing. Default: 0.05.
            diverge_th (int, optional): the test is stopped when the loss surpasses the
                threshold:  diverge_th * best_loss. Default: 5.
        """
        # Reset test results
        self.history = {"lr": [], "loss": []}
        self.best_loss = None

        # Move the model to the proper device
        self.model.to(self.device)

        # Initialize the proper learning rate policy
        if step_mode.lower() == "exp":
            lr_schedule = ExponentialLR(self.optimizer, end_lr, num_iter)
        elif step_mode.lower() == "linear":
            lr_schedule = LinearLR(self.optimizer, end_lr, num_iter)
        else:
            raise ValueError("expected one of (exp, linear), got {}".format(step_mode))

        if smooth_f < 0 or smooth_f >= 1:
            raise ValueError("smooth_f is outside the range [0, 1[")

        # Create an iterator to get data batch by batch
        iterator = iter(train_loader)
        for iteration in tqdm(range(num_iter)):
            # Get a new set of inputs and labels
            try:
                inputs, labels = next(iterator)
            except StopIteration:
                iterator = iter(train_loader)
                inputs, labels = next(iterator)

            # Train on batch and retrieve loss
            loss = self._train_batch(inputs, labels)
            if val_loader:
                loss = self._validate(val_loader)

            # Update the learning rate
            lr_schedule.step()
            self.history["lr"].append(lr_schedule.get_lr()[0])

            # Track the best loss and smooth it if smooth_f is specified
            if iteration == 0:
                self.best_loss = loss
            else:
                if smooth_f > 0:
                    loss = smooth_f * loss + (1 - smooth_f) * self.history["loss"][-1]
                if loss < self.best_loss:
                    self.best_loss = loss

            # Check if the loss has diverged; if it has, stop the test
            self.history["loss"].append(loss)
            if loss > diverge_th * self.best_loss:
                print("Stopping early, the loss has diverged")
                break

        print("Learning rate search finished. See the graph with {finder_name}.plot()")

    def _train_batch(self, inputs, labels):
        # Set model to training mode
        self.model.train()

        # Move data to the correct device
        inputs = inputs.to(self.device, dtype=torch.float)
        labels = labels.to(self.device, dtype=torch.float)

        # Forward pass
        self.optimizer.zero_grad()
        outputs = self.model(inputs)
        loss = self.criterion(outputs, labels)

        # Backward pass
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def _validate(self, dataloader):
        # Set model to evaluation mode and disable gradient computation
        running_loss = 0
        self.model.eval()
        with torch.no_grad():
            for inputs, labels in dataloader:
                # Move data to the correct device
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)

                # Forward pass and loss computation
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                running_loss += loss.item() * inputs.size(0)

        return running_loss / len(dataloader.dataset)

    def plot(self, skip_start=10, skip_end=5, log_lr=True):
        """Plots the learning rate range test.
        Arguments:
            skip_start (int, optional): number of batches to trim from the start.
                Default: 10.
            skip_end (int, optional): number of batches to trim from the start.
                Default: 5.
            log_lr (bool, optional): True to plot the learning rate in a logarithmic
                scale; otherwise, plotted in a linear scale. Default: True.
        """

        if skip_start < 0:
            raise ValueError("skip_start cannot be negative")
        if skip_end < 0:
            raise ValueError("skip_end cannot be negative")

        # Get the data to plot from the history dictionary. Also, handle skip_end=0
        # properly so the behaviour is the expected
        lrs = self.history["lr"]
        losses = self.history["loss"]
        if skip_end == 0:
            lrs = lrs[skip_start:]
            losses = losses[skip_start:]
        else:
            lrs = lrs[skip_start:-skip_end]
            losses = losses[skip_start:-skip_end]

        # Plot loss as a function of the learning rate
        plt.plot(lrs, losses)
        if log_lr:
            plt.xscale("log")
        plt.xlabel("Learning rate")
        plt.ylabel("Loss")
        plt.show()


class LinearLR(_LRScheduler):
    """Linearly increases the learning rate between two boundaries over a number of
    iterations.
    Arguments:
        optimizer (torch.optim.Optimizer): wrapped optimizer.
        end_lr (float, optional): the initial learning rate which is the lower
            boundary of the test. Default: 10.
        num_iter (int, optional): the number of iterations over which the test
            occurs. Default: 100.
        last_epoch (int): the index of last epoch. Default: -1.
    """

    def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
        self.end_lr = end_lr
        self.num_iter = num_iter
        super(LinearLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        curr_iter = self.last_epoch + 1
        r = curr_iter / self.num_iter
        return [base_lr + r * (self.end_lr - base_lr) for base_lr in self.base_lrs]


class ExponentialLR(_LRScheduler):
    """Exponentially increases the learning rate between two boundaries over a number of
    iterations.
    Arguments:
        optimizer (torch.optim.Optimizer): wrapped optimizer.
        end_lr (float, optional): the initial learning rate which is the lower
            boundary of the test. Default: 10.
        num_iter (int, optional): the number of iterations over which the test
            occurs. Default: 100.
        last_epoch (int): the index of last epoch. Default: -1.
    """

    def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
        self.end_lr = end_lr
        self.num_iter = num_iter
        super(ExponentialLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        curr_iter = self.last_epoch + 1
        r = curr_iter / self.num_iter
        return [base_lr * (self.end_lr / base_lr) ** r for base_lr in self.base_lrs]


class StateCacher(object):
    def __init__(self, in_memory, cache_dir=None):
        self.in_memory = in_memory
        self.cache_dir = cache_dir

        if self.cache_dir is None:
            import tempfile
            self.cache_dir = tempfile.gettempdir()
        else:
            if not os.path.isdir(self.cache_dir):
                raise ValueError('Given `cache_dir` is not a valid directory.')

        self.cached = {}

    def store(self, key, state_dict):
        if self.in_memory:
            self.cached.update({key: copy.deepcopy(state_dict)})
        else:
            fn = os.path.join(self.cache_dir, 'state_{}_{}.pt'.format(key, id(self)))
            self.cached.update({key: fn})
            torch.save(state_dict, fn)

    def retrieve(self, key):
        if key not in self.cached:
            raise KeyError('Target {} was not cached.'.format(key))

        if self.in_memory:
            return self.cached.get(key)
        else:
            fn = self.cached.get(key)
            if not os.path.exists(fn):
                raise RuntimeError('Failed to load state in {}. File does not exist anymore.'.format(fn))
            state_dict = torch.load(fn, map_location=lambda storage, location: storage)
            return state_dict

    def __del__(self):
        """Check whether there are unused cached files existing in `cache_dir` before
        this instance being destroyed."""
        if self.in_memory:
            return

        for k in self.cached:
            if os.path.exists(self.cached[k]):
                os.remove(self.cached[k])

/opt/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
dir_csv = '/mnt/disks/mount_point'
dir_train_img = '/mnt/disks/mount_point/stage_1_train_images_jpg'
dir_test_img = '/mnt/disks/mount_point/stage_1_test_images_jpg'

n_classes = 6
n_epochs = 1
batch_size = 9


import os
import cv2
import glob
#import pydicom
import numpy as np
import pandas as pd
from efficientnet_pytorch import EfficientNet
import torch
import torch.optim as optim
from albumentations import Compose, ShiftScaleRotate, Resize, CenterCrop, ToFloat
from albumentations.pytorch import ToTensor
from torch.utils.data import Dataset
from tqdm import tqdm_notebook as tqdm
#from torch_lr_finder import LRFinder
from sklearn.model_selection import train_test_split

In [5]:
class IntracranialDataset(Dataset):

    def __init__(self, csv_file, path, labels, transform=None):

        self.path = path
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.labels = labels

    def __len__(self):

        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.path, self.data.loc[idx, 'Image'] + '.jpg')
        img = cv2.imread(img_name)

        if self.transform:

            augmented = self.transform(image=img)
            img = augmented['image']

        if self.labels:

            labels = torch.tensor(
                self.data.loc[idx, ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']])
            #return {'image': img, 'labels': labels}
            return (img, labels)

        else:

            #return {'image': img}
            return img

In [6]:
transform_train = Compose([
    Resize(456, 456),
    ToFloat(max_value=127, always_apply=True),
    ShiftScaleRotate(),
    ToTensor()
])

transform_valid = Compose([
    Resize(456, 456),
    ToFloat(max_value=127, always_apply=True),
    ToTensor()
])

transform_test= Compose([
    Resize(456, 456),
    ToFloat(max_value=127, always_apply=True),
    ToTensor()
])

train_dataset = IntracranialDataset(
    csv_file='/mnt/disks/mount_point/train.csv', path='/mnt/disks/mount_point/stage_1_train_images_jpg', transform=transform_train, labels=True)

valid_dataset = IntracranialDataset(
    csv_file='/mnt/disks/mount_point/valid.csv', path='/mnt/disks/mount_point/stage_1_train_images_jpg', transform=transform_valid, labels=True)

test_dataset = IntracranialDataset(
    csv_file='/mnt/disks/mount_point/test.csv', path='/mnt/disks/mount_point/stage_1_test_images_jpg', transform=transform_test, labels=False)

data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
data_loader_valid = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
data_loader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [7]:
device = torch.device("cuda:0")
model = EfficientNet.from_pretrained('efficientnet-b5') 

Loaded pretrained weights for efficientnet-b5


In [8]:
model._fc = torch.nn.Linear(2048, n_classes)

In [9]:
model.to(device)
criterion = torch.nn.BCEWithLogitsLoss()
plist = [{'params': model.parameters(), 'lr': 5e-2}]
optimizer = optim.Adam(plist, lr=5e-2)

In [10]:
import time
import os
#i = 0
#train_csv_file = pd.read_csv(f'/mnt/disks/mount_point/train_splitted/train{i}.csv')

In [12]:
def training_model(i):
    
    train_dataset = IntracranialDataset(
        csv_file=f'/mnt/disks/mount_point/train_splitted/train{i}.csv', path='/mnt/disks/mount_point/stage_1_train_images_jpg', transform=transform_train, labels=True)
    print(f'/mnt/disks/mount_point/train_splitted/train{i}.csv')
    data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    since = time.time()
    
    #checkpoint = torch.load('/mnt/disks/mount_point/train{i-1}.pth')
    #model.load_state_dict(checkpoint['model_state_dict'])
    
    for epoch in range(n_epochs):
        print('Epoch {}/{}'.format(epoch, n_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            if phase == 'train':
                dataloader = data_loader_train
            else:
                dataloader = data_loader_valid
            for inputs, labels in dataloader:
                inputs = inputs.to(device, dtype=torch.float)
                labels = labels.to(device, dtype=torch.float)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #_, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                dataset_size = len(train_dataset)
            else:
                dataset_size = len(valid_dataset)
            epoch_loss = running_loss / dataset_size
            #epoch_acc = running_corrects.double() / dataset_size

            #print('{} Loss: {:.4f} Acc: {:.4f}'.format(
             #   phase, epoch_loss, epoch_acc))
            print(f'Loss: {epock_loss}')

            # deep copy the model
            #if phase == 'val' :
             #   val_acc = epoch_acc
              #  print(f'val_acc: {val_acc}')
                

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    torch.save(model.state_dict(), f'/mnt/disks/mount_point/train{i}.pth')
    return
    


In [3]:
ls /mnt/disks/mount_point/

example.pth                        stage_1_train.csv.zip
lost+found/                        stage_1_train_images_jpg/
process_csv.py                     stderr.txt
rsna_resnext.py                    stdout.txt
sample.txt                         test.csv
stage_1_sample_submission.csv      train.csv
stage_1_sample_submission.csv.zip  train_splitted/
stage_1_test_images_jpg/           trial.py
stage_1_train.csv                  valid.csv


In [20]:
!cat /mnt/disks/mount_point/stdout.txt

/mnt/disks/mount_point/train_splitted/train0.csv
Epoch 0/0
----------


In [21]:
!cat /mnt/disks/mount_point/stderr.txt

In [29]:
import sys
f1 = open('/mnt/disks/mount_point/stdout2.txt', mode='w')
f2 = open('/mnt/disks/mount_point/stderr2.txt', mode='w')
f3 = open('/mnt/disks/mount_point/richout.txt', mode='w')

In [14]:
from IPython.utils import io

In [ ]:
try:
    with io.capture_output() as captured:
        #for i in range(1):
         #   training_model(i)
        training_wrap()
finally:
    #os.system('sudo poweroff')
    with open('stdout3.txt', mode='w') as f:
        f.write(captured.stdout)
    with open('stderr3.txt', mode='w') as f:
        f.write(captured.stderr)
    with open('richout3.txt', mode='w') as f:
        f.write(str(out))
    os.system('sudo poweroff')

In [13]:
for i in range(1):
    training_model(i)

/mnt/disks/mount_point/train_splitted/train0.csv
Epoch 0/0
----------


KeyboardInterrupt: 

In [18]:
with io.capture_output() as captured:
    for i in range(1):
        training_model(i)
    with open('/mnt/disks/mount_point/stdout.txt', mode='w') as f:
        f.write(captured.stdout)
    with open('/mnt/disks/mount_point/stderr.txt', mode='w') as f:
        f.write(captured.stderr)
    with open('/mnt/disks/mount_point/richout.txt', mode='w') as f:
        f.write(captured.output)

NameError: name 'preds' is not defined

In [8]:
ls /mnt/disks/mount_point/

example.pth      sometext.txt                       test.csv
ipython_log.py   stage_1_sample_submission.csv      train.csv
lost+found/      stage_1_sample_submission.csv.zip  train_splitted/
ongoing.ipynb    stage_1_test_images_jpg/           trial.py
process_csv.py   stage_1_train.csv                  Untitled.ipynb
rsna.ipynb       stage_1_train.csv.zip              Untitled.py
rsna.py          stage_1_train_images_jpg/          valid.csv
rsna_resnext.py  stderr.txt
sample.txt       stdout.txt


In [31]:
!cat stdout.txt

/mnt/disks/mount_point/train_splitted/train0.csv
Epoch 0/0
----------


In [34]:
!cat richout.txt

In [15]:
import ipywidgets as widgets

In [16]:
out = widgets.Output(layout={'border': '1px solid black'})

In [38]:
out

Output(layout=Layout(border='1px solid black'))

In [17]:
@out.capture()
def training_wrap():
    for i in range(1):
        training_model(i)
        

In [40]:
training_wrap()

In [71]:
?out

Type:           Output
String form:    Output(layout=Layout(border='1px solid black'), outputs=({'name': 'stdout', 'text': '/mnt/disks/m <...> fined"], 'ename': 'NameError', 'evalue': "name 'preds' is not defined", 'output_type': 'error'}))
File:           /opt/anaconda3/lib/python3.7/site-packages/ipywidgets/widgets/widget_output.py
Docstring:     
Widget used as a context manager to display output.

This widget can capture and display stdout, stderr, and rich output.  To use
it, create an instance of it and display it.

You can then use the widget as a context manager: any output produced while in the
context will be captured and displayed in the widget instead of the standard output
area.

You can also use the .capture() method to decorate a function or a method. Any output 
produced by the function will then go to the output widget. This is useful for
debugging widget callbacks, for example.

Example::
    import ipywidgets as widgets
    from IPython.display import display
    out

In [43]:
with open('richout2.txt', mode='w') as f:
    f.write(str(out))

In [44]:
!cat richout2.txt

Output(layout=Layout(border='1px solid black'), outputs=({'name': 'stdout', 'text': '/mnt/disks/mount_point/train_splitted/train0.csv\nEpoch 0/0\n----------\n', 'output_type': 'stream'}, {'traceback': ['\x1b---------------------------------------------------------------------------\x1b', '\x1bNameError\x1b                                 Traceback (most recent call last)', '\x1b/opt/anaconda3/lib/python3.7/site-packages/ipywidgets/widgets/widget_output.py\x1b in \x1binner\x1b(*args, **kwargs)\x1b\n\x1b    101\x1b                     \x1bself\x1b\x1b.\x1b\x1bclear_output\x1b\x1b(\x1b\x1b*\x1b\x1bclear_args\x1b\x1b,\x1b \x1b**\x1b\x1bclear_kwargs\x1b\x1b)\x1b\x1b\x1b\x1b\x1b\x1b\n\x1b    102\x1b                 \x1bwith\x1b \x1bself\x1b\x1b:\x1b\x1b\x1b\x1b\x1b\x1b\n\x1b--> 103\x1b                     \x1breturn\x1b \x1bfunc\x1b\x1b(\x1b\x1b*\x1b\x1bargs\x1b\x1b,\x1b \x1b**\x1b\x1bkwargs\x1b\x1b)\x1b\x1b\x1b\x1b\x1b\x1b\n\x1b\x1b    104\x1b             \x1breturn\x1b \x1binner\x1b\x1b\x1

In [46]:
with open('richout2.txt', mode='r') as f:
    s = f.read()
    display(s)
    

'Output(layout=Layout(border=\'1px solid black\'), outputs=({\'name\': \'stdout\', \'text\': \'/mnt/disks/mount_point/train_splitted/train0.csv\\nEpoch 0/0\\n----------\\n\', \'output_type\': \'stream\'}, {\'traceback\': [\'\\x1b---------------------------------------------------------------------------\\x1b\', \'\\x1bNameError\\x1b                                 Traceback (most recent call last)\', \'\\x1b/opt/anaconda3/lib/python3.7/site-packages/ipywidgets/widgets/widget_output.py\\x1b in \\x1binner\\x1b(*args, **kwargs)\\x1b\\n\\x1b    101\\x1b                     \\x1bself\\x1b\\x1b.\\x1b\\x1bclear_output\\x1b\\x1b(\\x1b\\x1b*\\x1b\\x1bclear_args\\x1b\\x1b,\\x1b \\x1b**\\x1b\\x1bclear_kwargs\\x1b\\x1b)\\x1b\\x1b\\x1b\\x1b\\x1b\\x1b\\n\\x1b    102\\x1b                 \\x1bwith\\x1b \\x1bself\\x1b\\x1b:\\x1b\\x1b\\x1b\\x1b\\x1b\\x1b\\n\\x1b--> 103\\x1b                     \\x1breturn\\x1b \\x1bfunc\\x1b\\x1b(\\x1b\\x1b*\\x1b\\x1bargs\\x1b\\x1b,\\x1b \\x1b**\\x1b\\x1bkwargs\\x1b\\x

In [66]:
out2 = widgets.Output(layout={'border': '1px solid black'}, output=out.outputs)


In [67]:
out2

Output(layout=Layout(border='1px solid black'))

In [4]:
ls /mnt/disks/mount_point/

example.pth      sometext.txt                       stdout.txt
ipython_log.py   stage_1_sample_submission.csv      test.csv
lost+found/      stage_1_sample_submission.csv.zip  train.csv
process_csv.py   stage_1_test_images_jpg/           train_splitted/
rsna.ipynb       stage_1_train.csv                  trial.py
rsna.py          stage_1_train.csv.zip              Untitled.ipynb
rsna_resnext.py  stage_1_train_images_jpg/          Untitled.py
sample.txt       stderr.txt                         valid.csv


In [5]:
!cp /mnt/disks/mount_point/Untitled.py .